In [2]:
from datafin.aws import SecretsClient

s = SecretsClient()

In [16]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import io
from typing import Optional, Union, List


from datafin.aws import SecretsClient                      #type: ignore
from datafin.aws import S3Client                           #type: ignore

In [17]:
class GmailClient:
    """
    Enhanced Gmail client with support for PNG attachments
    """
    
    def __init__(self, sender_email: str, app_password: str):
        """
        Initialize Gmail client with credentials
        """
        self.sender_email = sender_email
        self.app_password = app_password
        self.smtp_server = "smtp.gmail.com"
        self.smtp_port = 587
        
        self._test_connection()
    
    def _test_connection(self) -> bool:
        """
        Test connection to Gmail SMTP server
        """
        try:
            server = smtplib.SMTP(self.smtp_server, self.smtp_port)
            server.starttls()
            server.login(self.sender_email, self.app_password)
            server.quit()
            return True
        except Exception as e:
            raise Exception(f"Failed to connect to Gmail SMTP: {str(e)}")
    
    def send_email(self, 
                   to: Union[str, List[str]], 
                   subject: str, 
                   text: str, 
                   df: Optional[pd.DataFrame] = None,
                   csv_filename: Optional[str] = None,
                   png_bytes: Optional[bytes] = None,
                   png_filename: Optional[str] = None,
                   cc: Optional[Union[str, List[str]]] = None,
                   bcc: Optional[Union[str, List[str]]] = None
        ) -> bool:
        """
        Send email with optional DataFrame CSV and PNG attachments
        
        Args:
            to: Recipient email(s)
            subject: Email subject
            text: Email body text
            df: Optional DataFrame to attach as CSV
            csv_filename: Optional CSV filename
            png_bytes: Optional PNG image bytes to attach
            png_filename: Optional PNG filename
            cc: Optional CC recipients
            bcc: Optional BCC recipients
        """
        try:
            msg = MIMEMultipart()
            msg['From'] = self.sender_email
            msg['To'] = ', '.join(to) if isinstance(to, list) else to
            msg['Subject'] = subject
            
            # Add CC and BCC headers if provided
            if cc:
                msg['Cc'] = ', '.join(cc) if isinstance(cc, list) else cc
            
            # Add email body
            msg.attach(MIMEText(text, 'plain'))
            
            # Attach DataFrame as CSV if provided
            if df is not None:
                self._attach_dataframe_as_csv(msg, df, csv_filename)
            
            # Attach PNG if provided
            if png_bytes is not None:
                self._attach_png(msg, png_bytes, png_filename)
            
            # Prepare recipient list
            recipients = []
            if isinstance(to, list):
                recipients.extend(to)
            else:
                recipients.append(to)
            
            if cc:
                if isinstance(cc, list):
                    recipients.extend(cc)
                else:
                    recipients.append(cc)
            
            if bcc:
                if isinstance(bcc, list):
                    recipients.extend(bcc)
                else:
                    recipients.append(bcc)
            
            # Send email
            server = smtplib.SMTP(self.smtp_server, self.smtp_port)
            server.starttls()
            server.login(self.sender_email, self.app_password)
            
            text_msg = msg.as_string()
            server.sendmail(self.sender_email, recipients, text_msg)
            server.quit()
            
            print(f"✓ Email sent successfully to {len(recipients)} recipient(s)")
            return True
            
        except Exception as e:
            raise Exception(f"Failed to send email: {str(e)}")
    
    def send_email_with_s3_png(self,
                               s3_client,
                               to: Union[str, List[str]],
                               subject: str,
                               text: str,
                               png_s3_path: str,
                               png_s3_filename: str,
                               png_attachment_filename: Optional[str] = None,
                               df: Optional[pd.DataFrame] = None,
                               csv_filename: Optional[str] = None,
                               cc: Optional[Union[str, List[str]]] = None,
                               bcc: Optional[Union[str, List[str]]] = None
        ) -> bool:
        """
        Send email with PNG attachment directly from S3
        
        Args:
            s3_client: Your S3Client instance
            to: Recipient email(s)
            subject: Email subject
            text: Email body text
            png_s3_path: S3 path where PNG is stored
            png_s3_filename: S3 filename (without .png extension)
            png_attachment_filename: Optional custom filename for attachment
            df: Optional DataFrame to attach as CSV
            csv_filename: Optional CSV filename
            cc: Optional CC recipients
            bcc: Optional BCC recipients
        """
        try:
            # Get PNG from S3
            png_bytes = s3_client.get_png(
                path=png_s3_path,
                file_name=png_s3_filename
            )
            
            # Use provided filename or default to S3 filename
            attachment_filename = png_attachment_filename or f"{png_s3_filename}.png"
            
            # Send email with PNG attachment
            return self.send_email(
                to=to,
                subject=subject,
                text=text,
                df=df,
                csv_filename=csv_filename,
                png_bytes=png_bytes,
                png_filename=attachment_filename,
                cc=cc,
                bcc=bcc
            )
            
        except Exception as e:
            raise Exception(f"Failed to send email with S3 PNG: {str(e)}")
    
    def _attach_dataframe_as_csv(self, msg: MIMEMultipart, df: pd.DataFrame, filename: Optional[str] = None) -> None:
        """
        Attach a pandas DataFrame as a CSV file to the email.
        
        Args:
            msg (MIMEMultipart): Email message object
            df (pd.DataFrame): DataFrame to attach
            filename (str, optional): CSV filename
        """
        if filename is None:
            filename = "data.csv"
        
        if not filename.endswith('.csv'):
            filename += '.csv'
        
        # Convert DataFrame to CSV string
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        csv_data = csv_buffer.getvalue()
        
        # Create attachment
        attachment = MIMEBase('application', 'octet-stream')
        attachment.set_payload(csv_data.encode())
        encoders.encode_base64(attachment)
        
        # Add header
        attachment.add_header(
            'Content-Disposition',
            f'attachment; filename= {filename}'
        )
        
        msg.attach(attachment)
        print(f"✓ DataFrame attached as {filename} ({len(df)} rows, {len(df.columns)} columns)")
    
    def _attach_png(self, msg: MIMEMultipart, png_bytes: bytes, filename: Optional[str] = None) -> None:
        """
        Attach PNG image bytes to the email.
        
        Args:
            msg (MIMEMultipart): Email message object
            png_bytes (bytes): Raw PNG image data
            filename (str, optional): PNG filename
        """
        if filename is None:
            filename = "image.png"
        
        if not filename.endswith('.png'):
            filename += '.png'
        
        # Create PNG attachment
        img_attachment = MIMEImage(png_bytes)
        img_attachment.add_header(
            'Content-Disposition',
            f'attachment; filename="{filename}"'
        )
        
        msg.attach(img_attachment)
        print(f"✓ PNG image attached as {filename} ({len(png_bytes)} bytes)")

In [ ]:
gmail_client = GmailClient(
        sender_email="your-email@gmail.com",
        app_password="your-app-password"
    )
    
    # Example 1: Send email with PNG from S3
    gmail_client.send_email_with_s3_png(
        s3_client=s3_client,  # Your S3Client instance
        to="recipient@example.com",
        subject="Daily Chart Report",
        text="Please find the attached candlestick chart for today's trading session.",
        png_s3_path="charts",
        png_s3_filename="candlestick_chart",
        png_attachment_filename="daily_chart.png"
    )
    
    # Example 2: Send email with both PNG and CSV
    gmail_client.send_email_with_s3_png(
        s3_client=s3_client,
        to=["recipient1@example.com", "recipient2@example.com"],
        subject="Trading Report with Chart and Data",
        text="Please find the attached chart and trading data.",
        png_s3_path="charts",
        png_s3_filename="candlestick_chart",
        df=your_dataframe,  # Your pandas DataFrame
        csv_filename="trading_data.csv",
        cc="manager@example.com"
    )

In [13]:
s3 = S3Client(
    aws_access_key_id=s.aws_access_key,
    aws_secret_access_key=s.aws_secret_access_key,
    bucket_name=s.get_bucket_name()
)

In [ ]:
image_bytes = fig.to_image(
    format="png",
    scale=4,
    engine="kaleido"
)

# Upload to S3
s3_client.post_png(
    image_bytes=image_bytes,
    path="charts",  # S3 folder path
    file_name="candlestick_chart"  # filename without extension
)

print("Chart uploaded to S3 successfully!")

'{"data": {"message": "connection to s3 bucket successful!"}}'